In [1]:
import torch
from Envs.simple_landmarks import MultiAgentLandmarks, MultiGoalEnv, MultiAgentLandmarksComm
from Envs.environment_handler import EnvironmentHandler
from agent import LSTMAgent
from policy import LSTM_PPO_Policy

In [2]:

env_config = {"num_landmarks": 3,
              "num_agents": 2,
              "timelimit": 10,
              "coop_chance":0.0,
              "message_len": 3,
              "vocab_size": 3,
              "message_penalty": 0.02,
              "seed": 42,
              "env_name": "MultiAgentLandmarks",
              "num_envs": 4}

model_config = {"channel_1": 16,
                "channel_2": 32,
                "channel_3": 32,
                "lstm_in_size": 256,
                "lstm_hidden_size": 256,
                "lstm_layers":1,
                }
config = {"env_config": env_config,
          "model": model_config,}


In [4]:
env = EnvironmentHandler(config)

In [5]:
obs, info = env.reset([i for i in range(4)])

In [6]:
for i in range(5):
    actions = torch.zeros((4, 2, 2))
    obs, rews, dones, infos = env.step(actions)
print(dones)

{'agent_0': array([False, False, False, False]), 'agent_1': array([False, False, False, False]), '__all__': array([False, False, False, False])}


In [12]:
def handle_dones(dones):
    """Handles the dones. Converts bools to binary tensor for each agent"""
    dones_out =  {}
    for key in dones.keys():
        if key == "__all__":
            continue
        else:
            dones_out[key] = torch.tensor(dones[key], dtype=torch.float32)
    return dones_out

In [13]:
next_dones = handle_dones(dones)
next_dones

{'agent_0': tensor([0., 0., 0., 0.]), 'agent_1': tensor([0., 0., 0., 0.])}

In [14]:
test = torch.zeros(4, 2)
ones = torch.ones(2)
test[[1, 2], [0, 0]] = ones
test

tensor([[0., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.]])

In [29]:
#Get dones
dones = torch.Tensor([0, 1, 1, 0])
obs = torch.randn(4, 2, 2)
#Get indexes
done_idx = dones.nonzero().squeeze()
print(done_idx)
#feed not done obs to network
input_obs = obs[done_idx]
fake_input_obs = torch.ones_like(input_obs)
#Instatiate next_obs tensor and broadcast obs to it
next_obs = torch.zeros(4, 2, 2)
next_obs[done_idx] = input_obs
print(next_obs)

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 1.9760,  1.4021],
         [ 0.2774,  0.2803]],

        [[-0.7494, -0.3849],
         [-1.7371,  0.9331]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]])


In [37]:
twos = torch.ones(2) * 2
ones = torch.ones(2)
condition = twos > ones
condition.all() 
if condition.all():
    print("All true")
ones.bool()

All true


tensor([True, True])

In [4]:
agent = LSTMAgent(env, config)
obs_shape = agent.observation_shape

In [5]:
input = torch.zeros((4, obs_shape[0], obs_shape[1], obs_shape[2]))
print(input.shape)

torch.Size([4, 64, 64, 3])


In [6]:
next_lstm_state = (
        torch.zeros(agent.lstm.num_layers, 4, agent.lstm.hidden_size),
        torch.zeros(agent.lstm.num_layers, 4, agent.lstm.hidden_size),
    ) 

done = torch.Tensor([0,0,0,0])

In [7]:
agent.get_action_and_value(input, next_lstm_state, done)

(tensor([[-0.2742, -2.1660],
         [-0.3977,  0.3407],
         [ 0.5400, -0.3248],
         [-0.6285, -0.8550]]),
 tensor([-4.2212, -1.9750, -2.0364, -2.4009], grad_fn=<SumBackward1>),
 tensor([2.8379, 2.8379, 2.8379, 2.8379], grad_fn=<SumBackward1>),
 tensor([[-0.0042],
         [-0.0042],
         [-0.0042],
         [-0.0042]], grad_fn=<AddmmBackward0>),
 (tensor([[[-0.0038, -0.0020, -0.0031,  ...,  0.0024, -0.0014, -0.0010],
           [-0.0038, -0.0020, -0.0031,  ...,  0.0024, -0.0014, -0.0010],
           [-0.0038, -0.0020, -0.0031,  ...,  0.0024, -0.0014, -0.0010],
           [-0.0038, -0.0020, -0.0031,  ...,  0.0024, -0.0014, -0.0010]]],
         grad_fn=<StackBackward0>),
  tensor([[[-0.0077, -0.0041, -0.0062,  ...,  0.0047, -0.0029, -0.0019],
           [-0.0077, -0.0041, -0.0062,  ...,  0.0047, -0.0029, -0.0019],
           [-0.0077, -0.0041, -0.0062,  ...,  0.0047, -0.0029, -0.0019],
           [-0.0077, -0.0041, -0.0062,  ...,  0.0047, -0.0029, -0.0019]]],
         gra

In [ ]:
policy = 